In [1]:
import pylangacq as pla

In [2]:
import numpy as np

In [10]:
import pandas as pd

In [37]:
import glob

In [61]:
import os

In [3]:
pla.__version__

'0.11.0'

In [4]:
# data = pla.read_chat('./Pitt/*.cha')

Data grab

In [4]:
cookie_data = pla.read_chat('./Pitt/*/cookie/*.cha')

In [5]:
sentence_data = pla.read_chat('./Pitt/*/sentence/*.cha')

In [63]:
sentence_fnames = glob.glob(os.getcwd()+'/Pitt/*/sentence/*.cha')

In [64]:
cookie_fnames = glob.glob(os.getcwd()+'/Pitt/*/cookie/*.cha')

Data exploration

In [6]:
len(cookie_data)

552

In [7]:
len(sentence_data)

240

In [9]:
cookie_data.number_of_utterances('PAR')

7054

In [52]:
# MMSE
cookie_data.headers()[fname]['Participants']['PAR']['education']

'30'

In [57]:
cookie_data.headers()[fname]['Participants']['PAR']

{'participant_name': 'Participant',
 'language': 'eng',
 'corpus': 'Pitt',
 'age': '58;',
 'sex': 'female',
 'group': 'Control',
 'SES': '',
 'participant_role': 'Participant',
 'education': '30',
 'custom': ''}

In [84]:
# fill missing mmse with group's mean mmse
def get_targets(dataset,filenames):
    
    targets = []
    groups = {}
    
    for fname in sorted(filenames):
        # PAR MMSE
#         print(fname)
        target = dataset.headers()[fname]['Participants']['PAR']['education']
        group = dataset.headers()[fname]['Participants']['PAR']['group']
        
        # to be replaced with group mean
        if target is '':
            targets.append(group)
            
        else:
            targets.append(int(target))
            if group in groups.keys():
                groups[group].append(int(target))
            else:
                groups[group] = [int(target)]
                
#     print('targets',targets)            
#     print('groups',groups)
    
    for g in groups.keys():
        m = np.mean(groups[g])
        groups[g] = int(m) if m % 1 > 5 else int(m) + 1
    
    #TODO replace with tensor
    targetseries = pd.Series(targets)
    
#     print('groups_mean',groups)
    
    for g in groups.keys():
        targetseries[[i for i,x in enumerate(targets) if x==g]] = groups[g]

    return targetseries

In [85]:
sentence_data_targets = get_targets(sentence_data,sentence_fnames)

In [87]:
cookie_data_targets = get_targets(cookie_data,cookie_fnames)

In [90]:
len(sentence_data_targets) == len(sentence_data)

True

In [91]:
len(cookie_data_targets) == len(cookie_data)

True